Note: Discussed with Animesh Sagar and Chaitanya Patil

In [1]:
import spacy
import wmd
import warnings
import en_core_web_sm
import textacy
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Task 1

In [3]:
# ref: https://github.com/dcavar/python-tutorial-for-ipython/blob/master/notebooks/spaCy%20Tutorial.ipynb
warnings.filterwarnings('ignore')

x=[]
nlp = spacy.load('en', create_pipeline=wmd.WMD.create_spacy_pipeline)
with open('ex1.txt', 'r') as f:
    data_read= f.readline()

seed_sentence = nlp("My poodle likes food in general.")
nlp = spacy.load('en_core_web_sm')
text = data_read
text_tokens = nlp(text)

for sentence in text_tokens.sents:
    x.append(nlp(sentence.text))

print("Sentence \t    Similarity Score")

for i in range(len(x)):
  sample=x[i]
  sample_no_stop = nlp(' '.join([str(t) for t in sample if not t.is_stop]))
  sample_pron = nlp(' '.join([str(t) for t in sample_no_stop if t.pos_ in ['NOUN', 'PROPN']] ))
  print( sample,  sample_pron.similarity(seed_sentence))

print("\nThe sentence in the text file most similar to seed sentence is :" , x[3])
 

Sentence 	    Similarity Score
This is a sentence. 0.40717865473987574
This is another sentence. 0.40717865473987574
John likes milk. 0.46725571131126653
Peter eats candy. 0.5662843523571846
My dog likes bones.
 0.460992989580924

The sentence in the text file most similar to seed sentence is : Peter eats candy.


# Task 2

In [0]:
text1=nlp(u'John loves apples')
text2=nlp(u'some man likes fruit')

In [5]:
warnings.filterwarnings('ignore')
distance_list = []

for token1 in text1:
    for token2 in text2:
        print(token1, token2, token1.similarity(token2))
        distance_list.append(token1.similarity(token2))

print(" \nMax distance_list found between 'loves' & 'likes' : ", max(distance_list))

John some 0.16244455
John man 0.27093855
John likes -0.016569503
John fruit 0.2554266
loves some 0.03814313
loves man 0.11364341
loves likes 0.5741299
loves fruit 0.03479158
apples some 0.017183943
apples man 0.15219189
apples likes 0.48223904
apples fruit 0.29982755
 
Max distance_list found between 'loves' & 'likes' :  0.5741299


In [6]:
# ref : https://www.python-course.eu/levenshtein_distance.php
def lev_dist(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([lev_dist(s[:-1], t)+1,
               lev_dist(s, t[:-1])+1, 
               lev_dist(s[:-1], t[:-1]) + cost])

    return res

print("Levenshtein distance between words : ", lev_dist("loves", "likes"))



Levenshtein distance between words :  2


In [7]:
import numpy as np

mean_dist=[]
for i in text1:
  for j in text2:
    mean_dist.append(lev_dist(str(i),str(j)))
    
print("The word mover's mean_dist is:" , np.mean(mean_dist))


The word mover's mean_dist is: 4.333333333333333


# Task 3

In [12]:
new_list=[]

SUBJ = ["nsubj","nsubjpass"] 
VERB = ["ROOT"] 
OBJ = ["dobj", "pobj"] 

with open('samplefile.txt', 'r') as f:
    data_my = f.readline()

final_sentence = nlp(data_my)

for i in final_sentence.sents:
    new_list.append(nlp(i.text))

def svo(sentence):
  text_my = nlp(str(sentence))
  sub_toks = [tok for tok in text_my if (tok.dep_ in SUBJ) ]
  obj_toks = [tok for tok in text_my if (tok.dep_ in OBJ) ]
  vrb_toks = [tok for tok in text_my if (tok.dep_ in VERB) ]
  return sub_toks, vrb_toks, obj_toks



nlist=[]
for i in range(len(new_list)):
  spacy_svo_output= (new_list[i], svo(str(new_list[i])))
  nlist.append(spacy_svo_output)
  print(spacy_svo_output)
  

(The Pentagon has suspended operational training for all Saudi military students in the United States, indefinitely halting flight instruction, firing range training and all other operations outside the classroom in the wake of a shooting last week at Naval Air Station Pensacola in Florida by a member of the Saudi Royal Air Force., ([Pentagon], [suspended], [training, students, States, instruction, training, classroom, wake, shooting, Pensacola, Florida, member, Force]))
(The suspension will affect nearly 900 Saudi students across the country, the Defense Department said on Tuesday., ([suspension, Department], [said], [students, country, Tuesday]))
(Classroom teaching, including language courses, will continue while Pentagon leaders review vetting procedures for all foreign military trainees., ([teaching, leaders], [continue], [courses, procedures, trainees]))
(An estimated 5,200 international students in the United States will be covered by the security review., ([students], [covered]

In [0]:
with open("output.txt","w+") as f:
    f.write(str(nlist))


# Steps

1. Install Spacy
2. Import sample text file and tokenize into sentences.
3. Print Subject, Verb, Object triplets if the word tokens exists in "nsubj", "nsubjpass", "Root", "pobj", "dobj".
4. Write the output into text file.